In [1]:
import sys  
sys.path.insert(0, '../')

from connection_to_db import connection

sys.path.insert(0, '/')

from preprocess import preprocess as preprocess_data

In [11]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [3]:
departures_test_name = 'departures_to_test'
departures_train_name = 'departures'

weather_test_name = 'weather_predict'
weather_train_name = 'weather'

In [4]:
_, conn, cur = connection()

In [5]:
def get_data(connection, table_name, weather_table_name, is_train=False):
    query = f'''
        SELECT * 
        FROM {table_name}
        LEFT JOIN {weather_table_name} ON {weather_table_name}.datetime = {table_name}.datetime_round
    '''
    if is_train:
        query += ' WHERE delay IS NOT NULL'
    data = pd.read_sql(query, connection)
    return data

In [6]:
def additional_preprocess(data):
    data = data.drop(columns=['scheduled_datetime', 'date']).fillna(0)
    return data

In [7]:
test = get_data(conn, departures_test_name, weather_test_name)
test = preprocess_data(test)
test = additional_preprocess(test)
test = test.drop(columns=['delay'])
test.head()

,carrier_code,flight,destination_airport,temp,humidity,precip,preciptype,snowdepth,windgust,windspeed,...,cloudcover,visibility,solarradiation,conditions,winspeed_greater_17,is_freezing_rain,is_first_day_half,weekday,is_weekend,season
0,51,7454,8,38.0,88.6,0.4,4,0.2,33.3,20.8,...,100.0,1.6,0.0,0,1,0,False,3,False,3
1,51,7452,8,38.0,88.6,0.4,4,0.2,33.3,20.8,...,100.0,1.6,0.0,0,1,0,False,3,False,3
2,51,7458,8,38.0,88.6,0.4,4,0.2,33.3,20.8,...,100.0,1.6,0.0,0,1,0,False,3,False,3
3,51,7456,8,38.0,88.6,0.4,4,0.2,33.3,20.8,...,100.0,1.6,0.0,0,1,0,False,3,False,3
4,16,6444,110,38.0,88.6,0.4,4,0.2,33.3,20.8,...,100.0,1.6,0.0,0,1,0,False,3,False,3


In [8]:
train = get_data(conn, departures_train_name, weather_train_name)
train = preprocess_data(train, is_data_train=True)
train.head()

,carrier_code,flight,destination_airport,scheduled_datetime,date,delay,temp,humidity,precip,preciptype,...,cloudcover,visibility,solarradiation,conditions,winspeed_greater_17,is_freezing_rain,is_first_day_half,weekday,is_weekend,season
0,68,3001,193,20:35:00,2022-07-16,0.0,70.2,78.5,0.0,2,...,97.4,9.9,7.0,5,0,0,False,5,True,1
1,68,3010,193,08:42:00,2022-07-16,0.0,68.1,89.8,0.0,2,...,100.0,8.3,83.0,5,0,0,True,5,True,1
2,68,3015,118,08:18:00,2022-07-16,0.0,68.1,89.8,0.0,2,...,100.0,8.3,83.0,5,0,0,True,5,True,1
3,68,3032,101,10:30:00,2022-07-16,0.0,70.3,83.8,0.0,2,...,99.4,8.9,287.0,5,0,0,True,5,True,1
4,68,3045,118,19:47:00,2022-07-16,0.0,71.2,75.7,0.0,2,...,96.4,9.9,35.0,5,0,0,False,5,True,1


In [109]:
def split_train(data):
    max_date = data['date'].max()
    train_old = data[data['date'] < max_date]
    train_new = data[data['date'] == max_date]
    return train_old, train_new

train_old, train_new = split_train(train)
train_old = additional_preprocess(train_old)
train_new = additional_preprocess(train_new)

In [110]:
bound = 60

In [111]:
y = train_old['delay'].to_numpy()
X = train_old.drop(columns=['delay']).to_numpy()

y_val = train_new['delay']
X_val = train_new.drop(columns=['delay']).to_numpy()

In [108]:
y = y > bound + 0
y_val = y_val > bound + 0

In [12]:
y.mean()

0.06546369081910142

In [65]:
y

array([False, False, False, ..., False,  True, False])

In [37]:
logreg = LogisticRegression(n_jobs=-1, class_weight={0:0.05, 1:0.95})
logreg.fit(X, y)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight={0: 0.05, 1: 0.95}, n_jobs=-1)

In [43]:
((logreg.predict_proba(X)[:, 1] > 0.7) == y)[~y].mean()

0.9819209226792165

In [44]:
logreg.predict_proba(X)[:, 1]

array([0.59627998, 0.62145197, 0.62234818, ..., 0.60769516, 0.63441976,
       0.64051958])

In [46]:
pos_count = y.sum()
pos_count

21136

In [79]:
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(np.concatenate((X[~y][:pos_count], X[y])), [False]*pos_count + [True]*pos_count)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(n_jobs=-1)

In [80]:
((logreg.predict_proba(np.concatenate((X[~y][:pos_count], X[y])))[:, 1] > 0.6) == \
 np.array([False]*pos_count + [True]*pos_count)).mean()

0.8043858819076457

169871    False
169872    False
169873    False
169874    False
169875    False
          ...  
170799    False
170800    False
170801    False
170802    False
170803    False
Name: delay, Length: 933, dtype: bool

In [95]:
((logreg.predict_proba(X_val)[:, 1] > 0.6) == y_val)[~y_val].mean()

0.27896512935883017

In [112]:
X_big = X[y > bound]
y_big = y[y > bound]
res_big = np.mean(y_big)

res_small = 0

In [117]:
y > bound

array([False, False, False, ..., False,  True, False])

In [119]:
preds = np.array([0] * len(y_val))
preds[(logreg.predict_proba(X_val)[:, 1] > 0.6)] = res_big

In [122]:
np.abs(preds - y_val).mean()

97.28081457663451